# Tugas Pemrograman Kecil 1
Mata Kuliah: Temu-Balik Informasi

Author: Luthfi Balaka

Dalam tugas ini, Anda akan mempelajari *text preprocessing* yang umum dilakukan dalam pengembangan sistem *Information Retrieval*. Terdapat tiga proses yang akan dibahas, yakni tokenisasi, *stemming*, dan *stop word removal*.

*Notebook* ini terdiri atas dua bagian: contoh kode dan soal. Bagian pertama akan menjadi landasan untuk Anda mengerjakan soal yang ada. Selamat mengerjakan dan semoga bermanfaat!

## Bagian 1: Contoh Kode

Bagian ini akan membahas mengenai contoh pemrosesan teks. Silakan pahami dan lengkapi beberapa fungsi untuk mengerjakan soal-soal di Bagian 2.

In [1]:
# Install package jika belum ada dan pastikan Anda menggunakan Python >= 3.7
# !pip install pandas==2.2.2
!pip install PySastrawi==1.2.0
!pip install nltk==3.8.1

In [2]:
# Import packages yang diperlukan
import re
import pandas as pd

from collections import defaultdict
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.Stemmer.CachedStemmer import CachedStemmer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

### 1.1: Tokenisasi

Tokenisasi adalah proses mengubah teks menjadi serangkaian token. Ada berbagai perspektif terkait apa yang dimaksud dengan token. Salah satu makna yang umum digunakan adalah setiap token merepresentasikan suatu kata tertentu. Namun, bisa juga token merepresentasikan informasi yang lebih granular seperti karakter.

Pada bagian ini, kita akan berfokus pada dua metode saja, yakni tokenisasi dengan *regular expression* (regex) dan Byte-Pair Encoding (BPE).

#### Metode 1: Tokenisasi dengan regex

In [3]:
def tokenize_text_regex(text: str):
    """
    Tokenisasi teks berdasarkan pola tokenizer_pattern
    """
    tokenizer_pattern = r'\b\w+\b'

    tokens = re.findall(tokenizer_pattern, text)

    return tokens


# Contoh tokenisasi pada suatu teks
text_1 = "Saya sedang mengerjakan tugas pada mata kuliah Temu-Balik Informasi."
tokens_by_regex_tokenizer = tokenize_text_regex(text_1)
print(f"Text: {text_1}")
print(f"Tokens of the text: {tokens_by_regex_tokenizer}")

Text: Saya sedang mengerjakan tugas pada mata kuliah Temu-Balik Informasi.
Tokens of the text: ['Saya', 'sedang', 'mengerjakan', 'tugas', 'pada', 'mata', 'kuliah', 'Temu', 'Balik', 'Informasi']


#### Metode 2: Tokenisasi BPE

Referensi:
- [Text Processing by Jurafsky](https://web.stanford.edu/~jurafsky/slp3/slides/2_TextProc_2023.pdf)
- [BPE Tutorial by HuggingFace](https://huggingface.co/learn/nlp-course/en/chapter6/5)

In [4]:
def get_word_freqs(corpus: list[str]):
    """
    Menghasilkan frekuensi dari tiap kata dengan format:
    {"kata1": f_1, "kata2": f_2, ...}
    """
    words = dict()
    for tokens in corpus:
      tokens = tokenize_text_regex(tokens)
      for token in tokens:
        new_token = token + "#"
        if new_token in words:
          words[new_token] += 1
        else:
          words[new_token] = 1

    return words


# Contoh pemanggilan
corpus = [
    "low low low low low lowest lowest newer newer newer",
    "newer newer newer wider wider wider new new",
    "Mahasiswa kelas Perolehan Informasi, termasuk saya, sedang mengerjakan tugas ini.",
    "Saya sedang mengerjakan tugas ini."
]

word_freqs = get_word_freqs(corpus)
print(f"Contoh word_freqs: {word_freqs}")

Contoh word_freqs: {'low#': 5, 'lowest#': 2, 'newer#': 6, 'wider#': 3, 'new#': 2, 'Mahasiswa#': 1, 'kelas#': 1, 'Perolehan#': 1, 'Informasi#': 1, 'termasuk#': 1, 'saya#': 1, 'sedang#': 2, 'mengerjakan#': 2, 'tugas#': 2, 'ini#': 2, 'Saya#': 1}


In [5]:
def get_initial_vocabulary(word_freqs: dict[str, int]):
    """
    Membuat vocabulary awal yang berisi karakter dari word_freqs
    """
    vocabulary = set()
    for word in word_freqs.keys():
        for letter in word:
            vocabulary.add(letter)
    vocabulary = sorted(vocabulary)
    return vocabulary


# Contoh pemanggilan
init_vocab = get_initial_vocabulary(word_freqs)
print(f"Contoh initial vocab: {init_vocab}")

Contoh initial vocab: ['#', 'I', 'M', 'P', 'S', 'a', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'r', 's', 't', 'u', 'w', 'y']


In [6]:
def compute_pair_freqs(word_splits: dict[str, list[str]], word_freqs: dict[str, int]):
    """
    Menghitung frekuensi tiap pair karakter
    """
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = word_splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs


# Contoh pemanggilan
word_splits: dict[str, list[str]] = {
    word: [c for c in word] for word in word_freqs.keys()
}
print(f"Word splits: {word_splits}")
pair_freqs = compute_pair_freqs(word_splits, word_freqs)
print(f"Contoh pair_freqs: {pair_freqs}")

Word splits: {'low#': ['l', 'o', 'w', '#'], 'lowest#': ['l', 'o', 'w', 'e', 's', 't', '#'], 'newer#': ['n', 'e', 'w', 'e', 'r', '#'], 'wider#': ['w', 'i', 'd', 'e', 'r', '#'], 'new#': ['n', 'e', 'w', '#'], 'Mahasiswa#': ['M', 'a', 'h', 'a', 's', 'i', 's', 'w', 'a', '#'], 'kelas#': ['k', 'e', 'l', 'a', 's', '#'], 'Perolehan#': ['P', 'e', 'r', 'o', 'l', 'e', 'h', 'a', 'n', '#'], 'Informasi#': ['I', 'n', 'f', 'o', 'r', 'm', 'a', 's', 'i', '#'], 'termasuk#': ['t', 'e', 'r', 'm', 'a', 's', 'u', 'k', '#'], 'saya#': ['s', 'a', 'y', 'a', '#'], 'sedang#': ['s', 'e', 'd', 'a', 'n', 'g', '#'], 'mengerjakan#': ['m', 'e', 'n', 'g', 'e', 'r', 'j', 'a', 'k', 'a', 'n', '#'], 'tugas#': ['t', 'u', 'g', 'a', 's', '#'], 'ini#': ['i', 'n', 'i', '#'], 'Saya#': ['S', 'a', 'y', 'a', '#']}
Contoh pair_freqs: defaultdict(<class 'int'>, {('l', 'o'): 7, ('o', 'w'): 7, ('w', '#'): 7, ('w', 'e'): 8, ('e', 's'): 2, ('s', 't'): 2, ('t', '#'): 2, ('n', 'e'): 8, ('e', 'w'): 8, ('e', 'r'): 13, ('r', '#'): 9, ('w', 'i'):

In [7]:
def merge_split(a: str, b: str, split: list[str]):
    """
    Melakukan merging a dan b pada split
    """
    # TODO: implementasi

    index = 1

    while index < len(split):
      if split[index-1] == a and split[index] == b:
        split =  split[:index-1] + [a+b] + split[index+1:]
        index -= 1
      else:
        index += 1

    return split

In [8]:
def merge_pair(a: str, b: str, word_splits: dict[str, list[str]]):
    """
    Merging a dan b pada word_splits
    """
    for word in word_splits.keys():
        split = word_splits[word]
        if len(split) > 1:
            word_splits[word] = merge_split(a, b, split)
    return word_splits

word_splits_new = merge_pair("h", "a", word_splits.copy())
print(f"Contoh word_splits sebelum merging: {word_splits}")
print(f"Contoh word_splits setelah merging: {word_splits_new}")

Contoh word_splits sebelum merging: {'low#': ['l', 'o', 'w', '#'], 'lowest#': ['l', 'o', 'w', 'e', 's', 't', '#'], 'newer#': ['n', 'e', 'w', 'e', 'r', '#'], 'wider#': ['w', 'i', 'd', 'e', 'r', '#'], 'new#': ['n', 'e', 'w', '#'], 'Mahasiswa#': ['M', 'a', 'h', 'a', 's', 'i', 's', 'w', 'a', '#'], 'kelas#': ['k', 'e', 'l', 'a', 's', '#'], 'Perolehan#': ['P', 'e', 'r', 'o', 'l', 'e', 'h', 'a', 'n', '#'], 'Informasi#': ['I', 'n', 'f', 'o', 'r', 'm', 'a', 's', 'i', '#'], 'termasuk#': ['t', 'e', 'r', 'm', 'a', 's', 'u', 'k', '#'], 'saya#': ['s', 'a', 'y', 'a', '#'], 'sedang#': ['s', 'e', 'd', 'a', 'n', 'g', '#'], 'mengerjakan#': ['m', 'e', 'n', 'g', 'e', 'r', 'j', 'a', 'k', 'a', 'n', '#'], 'tugas#': ['t', 'u', 'g', 'a', 's', '#'], 'ini#': ['i', 'n', 'i', '#'], 'Saya#': ['S', 'a', 'y', 'a', '#']}
Contoh word_splits setelah merging: {'low#': ['l', 'o', 'w', '#'], 'lowest#': ['l', 'o', 'w', 'e', 's', 't', '#'], 'newer#': ['n', 'e', 'w', 'e', 'r', '#'], 'wider#': ['w', 'i', 'd', 'e', 'r', '#'], 'n

In [9]:
def train_bpe(corpus: list[str], num_of_merges: int):
    """
    Melatih tokenizer BPE pada korpus untuk mendapatkan aturan merging
    yang akan digunakan untuk melakukan tokenisasi nantinya
    """
    merge_rules: list[tuple[str, str]] = []
    word_freqs: dict[str, int] = get_word_freqs(corpus)
    vocab: list[str] = get_initial_vocabulary(word_freqs)
    word_splits = {word: [c for c in word] for word in word_freqs.keys()}

    for i in range(num_of_merges):
        try:
            pair_freqs = compute_pair_freqs(word_splits, word_freqs)
            best_pair = ""
            max_freq = None

            for pair, freq in pair_freqs.items():
                if max_freq is None or max_freq < freq:
                    best_pair = pair
                    max_freq = freq

            merge_rules.append(best_pair)
            vocab.append("".join(best_pair))
            word_splits = merge_pair(
                best_pair[0], best_pair[1], word_splits
            )
        except:
            print(f"Iteration stops early at {i}")
            break
    return vocab, merge_rules

In [10]:
# Coba kita latih dengan teks pada corpus contoh
vocab, merge_rules = train_bpe(corpus, 10)

In [11]:
def tokenize_bpe(vocab: list[str], merge_rules: list[tuple[str, str]], text: str):
    """
    Melakukan tokenisasi pada corpus berdasarkan vocab dan merge_rules yang
    didapatkan dari proses training
    """
    tokenized_text = []
    for word in get_word_freqs([text]).keys():
        word_split: list[str] = []
        for char in word:
            if char in vocab:
                word_split.append(char)
            else:
                word_split.append("<UNK>")  # Karakter yang tidak dikenali

        for merge_rule in merge_rules:
            merge_str = "".join(merge_rule)
            i = 0
            while i < len(word_split) - 1:
                if (
                    word_split[i] == merge_rule[0]
                    and word_split[i + 1] == merge_rule[1]
                ):
                    word_split = word_split[:i] + [merge_str] + word_split[i + 2 :]
                else:
                    i += 1
        tokenized_text += word_split
    return tokenized_text

In [12]:
# Kita tes tokenisasi suatu teks
text_2 = "lower newer"
tokens_by_bpe_tokenizer = tokenize_bpe(vocab, merge_rules, text_2)
print(tokens_by_bpe_tokenizer)

['low', 'er#', 'newer#']


### 1.2: *Stemming*

*Stemming* merupakan proses transformasi kata dari bentuk infleksi ke bentuk dasar.
Secara umum, cara kerjanya adalah dengan "memotong" tambahan seperti afiks berdasarkan
aturan tertentu.

Contoh dalam Bahasa Indonesia: ["memakan", "dimakan", "termakan"] -> "makan"

Untuk task ini, kita akan menggunakan [PySastrawi](https://github.com/har07/PySastrawi).

In [13]:
# Mendefinisikan stemmer
factory: StemmerFactory = StemmerFactory()
stemmer: CachedStemmer = factory.create_stemmer()

In [14]:
def stem_tokens(stemmer: CachedStemmer, tokens: list[str]):
    """
    Melakukan stemming pada tokens
    """
    stemmed_tokens: list[str] = [
        stemmer.stem(token) if token else "" for token in tokens
    ]
    stemmed_tokens_without_empty_string: list[str] = [
        token for token in stemmed_tokens if not ((token == "") or (token == None))
    ]
    return stemmed_tokens_without_empty_string

In [15]:
# Contoh pemanggilan
stemmed_tokens = stem_tokens(stemmer, tokens_by_regex_tokenizer)
print(f"Kumpulan token sebelum stemming: {tokens_by_regex_tokenizer}")
print(f"Kumpulan token setelah stemming: {stemmed_tokens}")

Kumpulan token sebelum stemming: ['Saya', 'sedang', 'mengerjakan', 'tugas', 'pada', 'mata', 'kuliah', 'Temu', 'Balik', 'Informasi']
Kumpulan token setelah stemming: ['saya', 'sedang', 'kerja', 'tugas', 'pada', 'mata', 'kuliah', 'temu', 'balik', 'informasi']


### 1.3: *Stop Words Removal*

*Stop word* merupakan kata-kata yang umumnya memiliki frekuensi yang sangat tinggi dalam teks namun tidak memberikan informasi yang signifikan. Oleh karena itu, kata-kata tersebut seringkali dihapus
agar menyisakan informasi yang "penting" saja.

Contoh *stop words* pada Bahasa Indonesia: "yang", "di", "pada"

Kita akan menggunakan PySastrawi juga untuk *task* ini.

In [16]:
# Mendefinisikan kumpulan stop words
stop_factory = StopWordRemoverFactory()
stop_words = set(stop_factory.get_stop_words())

In [17]:
def remove_stop_words(tokens: list[str], stop_words: set[str]):
    """
    Menghapus stop words dari kumpulan token
    """
    tokens_without_stop_words = [token for token in tokens if token not in stop_words]
    return tokens_without_stop_words

In [18]:
# Contoh pemanggilan
stop_words_removed_tokens = remove_stop_words(stemmed_tokens, stop_words)
print(f"Kumpulan token sebelum stop words dihapus: {stemmed_tokens}")
print(f"Kumpulan token setelah stop words dihapus: {stop_words_removed_tokens}")

Kumpulan token sebelum stop words dihapus: ['saya', 'sedang', 'kerja', 'tugas', 'pada', 'mata', 'kuliah', 'temu', 'balik', 'informasi']
Kumpulan token setelah stop words dihapus: ['tugas', 'kuliah', 'temu', 'informasi']


### 1.4: Kombinasi 1.1 - 1.3

Semua fungsi di atas bisa dirangkai menjadi suatu *pipeline*.

In [19]:
# Contoh melakukan pemrosesan menggunakan regex tokenizer; Anda bisa gunakan BPE sebagai alternatif
def text_processing_pipeline(
    text: str, stemmer: CachedStemmer, stop_words: set[str]
):
    tokens = tokenize_text_regex(text)
    tokens = stem_tokens(stemmer, tokens)
    tokens = remove_stop_words(tokens, stop_words)
    return tokens

In [20]:
# Contoh menjalankan pipeline
tokens_by_pipeline = text_processing_pipeline(
    text_1, stemmer, stop_words
)
print(f"Teks: {text_1}")
print(f"Tokens: {tokens_by_pipeline}")

Teks: Saya sedang mengerjakan tugas pada mata kuliah Temu-Balik Informasi.
Tokens: ['tugas', 'kuliah', 'temu', 'informasi']


Hasil akhir dari penerapan tersebut adalah data (tokens) yang siap digunakan untuk pemrosesan selanjutnya dalam aplikasi IR.

## Bagian 2: Soal

Pada bagian ini, Anda diminta untuk menerapkan apa yang sudah Anda pelajari pada
Bagian 1. Dalam menjawab pertanyaan, Anda dibebaskan untuk menambah *cell* baru sesuai kebutuhan.
Selain itu, Anda juga dibebaskan untuk menambahkan *package* lain jika dibutuhkan
(pastikan untuk menuliskan *package* yang digunakan dan versinya).

In [21]:
# Install package lain jika perlu
# Saya sudah menginstall versi nltk 3.8.1 (dipaling atas)
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

nltk.download('stopwords')
english_stopwords = stopwords.words('english')
print(english_stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
# Load dataset yang diadaptasi dari https://huggingface.co/datasets/roneneldan/TinyStories
df = pd.read_csv("stories.csv")

# Gunakan korpus berikut untuk pengerjaan soal
corpus = df["story"]

corpus

,story
0,"Spot. Spot saw the shiny car and said, ""Wow, K..."
1,"Once upon a time, in a big forest, there lived..."
2,"Once upon a time, in a small yard, there was a..."
3,"Once upon a time, there was a thoughtful girl ..."
4,"Once upon a time, there was a kind farmer. He ..."
...,...
2995,"One day, Tom went on a search. Tom wanted to f..."
2996,Once upon a time there was a careless boy call...
2997,"One day, a little boy was walking through the ..."
2998,Once there was a girl called Daisy. Daisy was ...


### Soal 1: Menyesuaikan Komponen *Preprocessing* untuk Bahasa Inggris

Perhatikan bahwa `corpus` yang digunakan berbahasa Inggris. Oleh karena itu, beberapa
komponen dari Bagian 1.1 - 1.3 perlu disesuaikan. Silakan Anda pertimbangkan apa
saja yang perlu diubah dan apa yang bisa digunakan untuk Bahasa Inggris juga.
Implementasikan kodenya.

In [23]:
from typing import Any, Optional

def stem_english_tokens(tokens: list[str], stemmer: Any):
  stemmed_tokens: list[str] = [
        stemmer.stem(token) if token else "" for token in tokens
    ]
  stemmed_tokens_without_empty_string: list[str] = [
      token for token in stemmed_tokens if not ((token == "") or (token == None))
  ]
  return stemmed_tokens_without_empty_string

def remove_english_stop_words(tokens: list[str], stop_words: list[str]):
    tokens_without_stop_words = [token for token in tokens if token not in stop_words]
    return tokens_without_stop_words

### Soal 2: Membandingkan *Pipeline* Berbasiskan Regex dan BPE

Dengan komponen yang sudah Anda sesuaikan pada Soal 1, implementasikan dua *pipeline*
dengan *tokenizer* berbeda (antara regex atau BPE). Untuk BPE, latih pada `corpus`
dan gunakan suatu nilai `num_of_merges`. Jelaskan alasan Anda menggunakan nilai
`num_of_merges` tersebut. Lalu, jalankan kedua *pipeline* pada tiga teks yang dipilih
secara acak dari `corpus`. Analisis kelebihan dan kekurangan dari masing-masing
*pipeline* pada ketiga teks tersebut.

In [24]:
ps = PorterStemmer()

def english_text_processing_pipeline_regex(
    text: str, stemmer: Optional[Any] = ps, english_stopwords: Optional[list[str]] = english_stopwords
):
    tokens = tokenize_text_regex(text)
    tokens = stem_english_tokens(tokens, stemmer)
    tokens = remove_english_stop_words(tokens, english_stopwords)
    return tokens

def english_text_processing_pipeline_bpe(
    text: str, vocab: list[str], merge_rules: list[tuple[str, str]], stemmer: Optional[Any] = ps, english_stopwords: Optional[list[str]] = english_stopwords
):
    tokens = tokenize_bpe(vocab, merge_rules, text)
    tokens = stem_english_tokens(tokens, stemmer)
    tokens = remove_english_stop_words(tokens, english_stopwords)

    return tokens

# Pick 3 randomly
random_corpus = corpus.sample(n=3, random_state=42)
random_stories = list(random_corpus)
print(random_corpus)

print("Regex Pipeline")
print(40*"=")
for idx in range(len(random_stories)):
  text = random_stories[idx]
  pipeline_tokens = english_text_processing_pipeline_regex(
    text, ps, english_stopwords
  )
  print(f"Teks {idx + 1}: {text}")
  print(f"Tokens {idx + 1}: {pipeline_tokens}")
  print()

print()
print("BPE Pipeline")
print(40*"=")
vocab, merge_rules = train_bpe(corpus, 1000)
# BPE With 10 merge rules with vocab.

for index in range(len(random_stories)):
  text = random_stories[index]
  pipeline_tokens = english_text_processing_pipeline_bpe(
    text, vocab, merge_rules, ps, english_stopwords
  )
  print(f"Teks {index + 1}: {text}")
  print(f"Tokens {index + 1}: {pipeline_tokens}")
  print()

1801    Once there was a noisy alligator. He liked to ...
1190    Once upon a time, there was a little boy named...
1817    Timmy was a very obedient boy. He was always d...
Name: story, dtype: object
Regex Pipeline
Teks 1: Once there was a noisy alligator. He liked to cause a lot of commotion and he frequently made loud noises. One day, he was walking through the swamp, loudly singing and causing a ruckus.
He came across a duck who said to him, “Please be quiet! You’re being very noisy.” But the alligator refused to listen and kept singing.
He walked a bit further and came across a frog who said the same thing to him. “Please don’t be so noisy!” But the alligator continued to sing and be disruptive.
Finally, the alligator came across a wise old turtle and she said to him, “It’s nice to be happy, but it’s not nice to be too loud. You’d be better off if you kept your voice down and behaved more politely.”
The alligator heeded the turtle’s advice and from then on he made sure to stay qui

In [25]:
regex_size = 0

for story in corpus:
  regex_size += len(tokenize_text_regex(story))

print("Total words (counting duplicate) in corpus: ", regex_size)

Total words (counting duplicate) in corpus:  491612


**Memilih number of merge**
Menurut saya, pemilihan `number_of_merge` dapat bergantung dari beberapa karakteristik berikut:
- Ukuran corpus: apabila ukuran corpus kecil, maka number_of_merge hanya memerlukan number_of_merge yang kecil untuk hasilnya yang baik.
- Namun dari ukuran corpus tersebut, memang apabila number_of_merge terlalu besar maka dapat menyebabkan BPE tokenizer kita menjadi **overfitting**, Hal ini dapat mengakibatkan penurunan performa pada data yang belum pernah dilihat sebelumnya.


Saya memilih n = 1000, karena artinya dapat dilihat bahwa maksimal kata yang ada dicorpus yang akan kita latih adalah 491612 ribu. Nah, apabila kita menggunakan n yang terlalu besar misalnya di sekitar 300000 maka kemungkinan besar, tokenizer kita akan terlalu overfitting terhadap corpus kita. Oleh karena itu saya memilih 1000 karena untuk number of merge 1000, tokenizer kita dapat melihat pola-pola subword yang didapat.


**Kelebihan dan kekurangan**

Dapat dilihat dari hasil random corpus yang diperoleh, apabila kita menggunakan pipeline dengan step yang sama, contohnya pada kedua tipe tersebut tahap meliputi tokenizer (regex/BPE) -> stemming word -> stopwords removal terdapat beberapa kelebihan dan kekurangan yang dapat saya observasi:

Untuk Regex Tokenizer:
  
  **Kelebihan:**
  - Berdasarkan observasi saya **cepat dan sederhana**, untuk setiap story pada corpus yang ada, regex tokenizer adalah tokenizer yang sangat cepat. Hal ini karena regex tokenizer sendiri bersifat naive dan tidak mementingkan pola yang ada selain apabila itu adalah suatu alphanumerik, maka akan termasuk 1 token. Hal ini tentunya kurang efisien untuk kata-kata yang lebih kompleks seperti Rumah Sakit dimana artinya bukanlah rumah dan sakit melainkan 1 kali representasi.
  - Observasi saya selanjutnya, **dari segi fleksibilitas**, dengan menggunakan regex, kita dapat menentukan definisi token menurut kita. Jadi regex dapat menjadi satu cara yang powerful apabila kita ingin mendefinisikan token kita sendiri
  - Terakhir, **mudah dipahami**, tokenizer adalah hal yang cukup mudah dipahami. Simplenya, tokenizer regex adalah token-token yang diperoleh berdasarkan pola match yang ada.

**Kekurangan:**
- Berdasarkan observasi yang saya dapatkan, regex tokenizer masih memiliki **kekurangan dalam prsesisi dalam menangani kata-kata yang kompleks**
- Selanjutnya, regex tokenizer masih **tidak menggunakan frekuensi atau data dari text yang ada**, hal ini tentu saja masalah ketika kata-kata sedikit yang sering muncul bersamaan namun dianggap sebagai token yang berbeda.
- Apabila bahasa yang berbeda-beda dan tidak alfabet seperti biasanya seperti (mandarin, arab, dll), maka regex tokenizer kemungkinan besar tidak dapat bekerja sesuai dengan ekspektasi kita.


Untuk BPE:

**Kelebihan dari BPE**
- Dengan *n* yang besar, BPE **dapat mengurangi ukuran vocabulary** dengan menggabungkan sub-kata yang sering muncul bersamaan.
- BPE bekerja lebih baik untuk bahasa yang memiliki banyak bentuk kata karena dapat memecah kata menjadi sub-kata yang lebih kecil, memungkinkan untuk menangani kata-kata baru yang belum pernah dilihat sebelumnya.
- BPE dapat dilatih berdasarkan corpus sedangkan regex tokenizer tidak perlu pelatihan, hal ini merupakan kelebihan karena BPE termasuk data driven tokenizer yang sesuai dengan pola dari corpus yang kita miliki.

**Kekurangan dari BPE**
- BPE pada umumnya menghasilkan token yang lebih panjang karena BPE memecah kata menjadi sub-kata, hasil tokenisasi biasanya menghasilkan lebih banyak token dibandingkan dengan regex tokenizer, yang dapat memperpanjang waktu pemrosesan dan mengonsumsi lebih banyak memori.
- Selanjutnya efektivitas tokenisasi BPE sangat tergantung pada data pelatihan. Jika data pelatihan tidak mencakup variasi kata yang cukup, hasil tokenisasi bisa kurang optimal.

### Soal 3: Membangun *Stop Words*

Dengan memanfaatkan *tokenizer* regex, Anda diminta untuk mengumpulkan seluruh
token unik beserta frekuensi kemunculannya dari `corpus`. Berdasarkan frekuensi
kemunculan ini, ambil *top-200* token yang memiliki frekuensi tertinggi sebagai
*stop words*.

Tampilkan *stop words* Anda dan bandingkan dengan *stop words* yang Anda gunakan
di Soal 1 dan 2. Jelaskan hasil observasi Anda.

In [26]:
# TODO: kode

word_freqs = get_word_freqs(corpus)

sorted_word_freqs = sorted(word_freqs.items(), key=lambda x:x[1], reverse = True)

print("Top 200 kata dengan frekuensi:", sorted_word_freqs)

stop_words_word_freqs = [word[:-1] for word, freq in sorted_word_freqs[:200]]

print("Top 200 kata: ", sorted(stop_words_word_freqs))
print("Stopwords: ", sorted(english_stopwords))

total = 0
total_frequency = 0

for word in stop_words_word_freqs:
  if (word in english_stopwords):
    total += 1
    temp_word = word + "#"
    total_frequency += word_freqs[temp_word]

print("Total frequency of stopwords in corpus:", total_frequency)

print(f"Total number of the same stopwords:", total)

Top 200 kata dengan frekuensi: [('and#', 22266), ('the#', 21583), ('to#', 17076), ('a#', 15214), ('was#', 13866), ('it#', 7277), ('her#', 6303), ('He#', 5665), ('She#', 5545), ('he#', 5428), ('The#', 4923), ('she#', 4507), ('said#', 4318), ('in#', 4204), ('his#', 4154), ('They#', 4052), ('that#', 3937), ('with#', 3910), ('Lily#', 3906), ('day#', 3894), ('so#', 3691), ('of#', 3586), ('had#', 3280), ('they#', 3066), ('on#', 3051), ('s#', 2968), ('time#', 2830), ('little#', 2818), ('there#', 2739), ('for#', 2707), ('very#', 2490), ('you#', 2457), ('t#', 2323), ('I#', 2313), ('happy#', 2278), ('Once#', 2276), ('mom#', 2273), ('saw#', 2230), ('It#', 2211), ('One#', 2204), ('but#', 2136), ('big#', 2099), ('upon#', 2008), ('were#', 1936), ('girl#', 1912), ('play#', 1851), ('him#', 1832), ('Timmy#', 1746), ('wanted#', 1734), ('not#', 1631), ('up#', 1630), ('went#', 1563), ('is#', 1491), ('all#', 1486), ('be#', 1393), ('them#', 1387), ('But#', 1375), ('loved#', 1368), ('named#', 1352), ('friend

Dari stopwords yang saya dapatkan dari NLTK dibandingkan top 200 frekuensi yang saya peroleh dari tokenizer regex, dapat dilihat bahwa terdapat 72 dari 200 top frequency data yang cocok dengan stopwords yang saya download dari package NLTK.

Artinya stopwords mendominasi 36% besar dari top 200 kata yang ada pada corpus. Selanjutnya, apabila kita lihat bahwa dalam korpus sendiri (dapat dilihat pada jawaban no 2 pada analisis, total kata yang dimiliki adalah sebesar 491612 dan jumlah stopwords yang ada pada corpus adalah 204890). Dapat dilihat bahwa 42 % dari corpus adalah stopwords.